In [28]:
import os, ssl, certifi

# Force all libs (httpx/requests/groq) to use certifi bundle
os.environ["SSL_CERT_FILE"] = certifi.where()
os.environ["REQUESTS_CA_BUNDLE"] = certifi.where()
ssl._create_default_https_context = lambda: ssl.create_default_context(cafile=certifi.where())


In [29]:
import requests, certifi, os

os.environ["SSL_CERT_FILE"] = certifi.where()
os.environ["REQUESTS_CA_BUNDLE"] = certifi.where()

print("Certifi bundle:", certifi.where())
print("Google test:", requests.get("https://www.google.com").status_code)


Certifi bundle: c:\Users\abhav\OneDrive\Desktop\AgenticAi\.venv\Lib\site-packages\certifi\cacert.pem
Google test: 200


In [30]:
from langchain_groq import ChatGroq
from dotenv import load_dotenv
import os

load_dotenv()

model = ChatGroq(
    model="llama-3.1-8b-instant",
    api_key=os.getenv("GROQ_API_KEY"),
)

print("Model ready ✅", model)


Model ready ✅ client=<groq.resources.chat.completions.Completions object at 0x00000167710DD010> async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x00000167710DDBE0> model_name='llama-3.1-8b-instant' model_kwargs={} groq_api_key=SecretStr('**********')


### Example 1: Simple LLM call with streaming

In [31]:
from langchain.chat_models import init_chat_model
from langchain_core.messages import HumanMessage,SystemMessage

In [32]:
from langchain_groq import ChatGroq

model = ChatGroq(
    model="llama-3.1-8b-instant",
    api_key=os.getenv("GROQ_API_KEY")
)
print(model)


client=<groq.resources.chat.completions.Completions object at 0x0000016773674550> async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000016773674F50> model_name='llama-3.1-8b-instant' model_kwargs={} groq_api_key=SecretStr('**********')


In [33]:
from langchain_core.messages import HumanMessage, SystemMessage

In [35]:
# create message 
messages = [
    SystemMessage("You are a helpful AI assistant"),
    HumanMessage("What eare the top 2 benefit using Langchain")
]

In [36]:
# invoke the model
response =  model.invoke(messages)
response

AIMessage(content="Langchain is an open-source framework for building conversational AI models. Based on my knowledge, here are the top 2 benefits of using Langchain:\n\n1. **Improved Conversational Flow**: Langchain allows developers to create more natural-sounding conversations by seamlessly integrating multiple AI models and knowledge sources. This enables Langchain to provide more accurate and context-aware responses, making conversations feel more human-like.\n\n2. **Increased Knowledge Retrieval and Reasoning**: Langchain's ability to leverage multiple knowledge sources and AI models enables it to perform more complex tasks, such as question answering, text summarization, and even creative writing. By combining the strengths of different models and sources, Langchain can provide more accurate and informative responses to a wide range of questions and prompts.\n\nThese benefits make Langchain a valuable tool for developers and researchers working on conversational AI projects.", a

In [37]:
print(response.content)

Langchain is an open-source framework for building conversational AI models. Based on my knowledge, here are the top 2 benefits of using Langchain:

1. **Improved Conversational Flow**: Langchain allows developers to create more natural-sounding conversations by seamlessly integrating multiple AI models and knowledge sources. This enables Langchain to provide more accurate and context-aware responses, making conversations feel more human-like.

2. **Increased Knowledge Retrieval and Reasoning**: Langchain's ability to leverage multiple knowledge sources and AI models enables it to perform more complex tasks, such as question answering, text summarization, and even creative writing. By combining the strengths of different models and sources, Langchain can provide more accurate and informative responses to a wide range of questions and prompts.

These benefits make Langchain a valuable tool for developers and researchers working on conversational AI projects.


In [41]:
model.invoke([HumanMessage("What are the top 2 benefits of lanchain?")])

AIMessage(content='I couldn\'t find any specific information on "lanchain." However, I can provide information on "Lanchain," which appears to be a decentralized identity verification and authentication project. If that\'s the case, here are two potential benefits of Lanchain:\n\n1. **Improved Data Security**: Lanchain is built on blockchain technology, which provides a secure and decentralized way to store and manage identity data. This helps prevent data breaches and unauthorized access, ensuring that users\' personal information remains safe.\n\n2. **Interoperability**: Lanchain\'s goal is to provide a common standard for identity verification across different platforms and industries. By doing so, it enables seamless and secure identity verification, reducing the need for multiple login processes and increasing the overall user experience.\n\nPlease note that these benefits are speculative and based on general assumptions, as I couldn\'t find any concrete information on Lanchain. I

In [43]:
## streaming example
for chunk in model.stream(messages):
    print(chunk.content, end = "", flush=True)

LangChain is an open-source framework for building large language models. Some of the key benefits of using LangChain include:

1. **Efficient Model Integration**: LangChain allows developers to integrate multiple large language models in a seamless manner. This enables developers to combine the strengths of different models to build more robust and accurate applications. By integrating multiple models, developers can leverage the individual strengths of each model to achieve better performance and accuracy in their applications.

2. **Flexibility and Customization**: LangChain provides a flexible framework that enables developers to customize and fine-tune their models to suit specific use cases and applications. This flexibility allows developers to create tailored solutions that meet the unique needs of their projects. Additionally, LangChain's modular architecture makes it easy to swap out different models or components as needed, allowing developers to quickly adapt to changing re

## Dynamic Prompt Template

In [45]:
from langchain_core.prompts import ChatPromptTemplate

## create translation app 
translation_template = ChatPromptTemplate.from_messages([
    ("system","You are a professional translator. Translate the following text {text} to {source_language} to {target_language}"),
    ("user","{text}")
])

# using the template
prompt = translation_template.invoke({
    "source_language":"English",
    "target_language":"French",
    "text":"Langchain makes building AI application incredible easy!"
})

In [46]:
prompt

ChatPromptValue(messages=[SystemMessage(content='You are a professional translator. Translate the following text Langchain makes building AI application incredible easy! to English to French', additional_kwargs={}, response_metadata={}), HumanMessage(content='Langchain makes building AI application incredible easy!', additional_kwargs={}, response_metadata={})])

In [48]:
translated_response = model.invoke(prompt)
translated_response.content

"Langchain rend la création d'applications d'intelligence artificielle incroyablement facile."

## Building our first chain


In [57]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableLambda

def create_story_chain():
    # Template for story generation
    story_prompt = ChatPromptTemplate.from_messages([
        ('system','You are a creative storyteller. Write a short and engaging story based on a given theme, character and settings'),
        ('user',"Theme: {theme}\n Main character: {character}\n Setting:{setting}")
    ])

    
    # template  for story analysis 
    analysis_prompt = ChatPromptTemplate.from_messages([
        ('system',"You are a literary critic. Analyze tthe following story and provide insights."),
        ('user',"{story}")
    ])

    # create a function to pass the story to analysis
    def analyze_story(story_text):
        return {'story':story_text}


    story_chain = (story_prompt | model | StrOutputParser())
    analysis_chain = (
        story_chain
        | RunnableLambda(analyze_story )
                      | analysis_prompt | model | StrOutputParser()
                      )
    
    return analysis_chain

In [58]:
chain= create_story_chain()
chain

ChatPromptTemplate(input_variables=['character', 'setting', 'theme'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a creative storyteller. Write a short and engaging story based on a given theme, character and settings'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['character', 'setting', 'theme'], input_types={}, partial_variables={}, template='Theme: {theme}\n Main character: {character}\n Setting:{setting}'), additional_kwargs={})])
| ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000016773674550>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000016773674F50>, model_name='llama-3.1-8b-instant', model_kwargs={}, groq_api_key=SecretStr('**********'))
| StrOutputParser()
| RunnableLambda(analyze_story)
| ChatPromptTemplate(input_variables=['story'], input_ty

In [59]:
result = chain.invoke({
    "theme":"artificial intelligence",
    "character":"a curious robot",
    "setting":"a futuristic city"
})

print("Story and Analysis")
print(result)

Story and Analysis
**Analyzing the Story: A Commentary on Humanity, AI, and Existentialism**

This short story, set in the futuristic city of Neo-Tokyo, presents a compelling narrative that explores the intersection of humanity and artificial intelligence. Through the protagonist, Zeta, a small, curious robot, the story delves into the complexities of existence, free will, and the nature of consciousness.

**The Labyrinth of Steel and Neon Lights: A Metaphor for Human Existence**

The opening paragraph sets the tone for the story, describing Neo-Tokyo as a "labyrinth of steel and neon lights." This metaphor suggests that human existence is a complex, ever-changing tapestry, navigated by individuals who must adapt and evolve to survive. The city, a symbol of human innovation and progress, is also a reflection of the chaos and uncertainty that accompany human existence.

**Zeta: A Prototype of Existential Crisis**

Zeta, the protagonist, is a thought-provoking character who embodies the 